In [ ]:
import numpy as np
import pandas as pd
from fredapi import Fred
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX  

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:

fred = Fred(api_key='321467cef92af0c44aa2aaf01257acc5')
cpi = fred.get_series('CPIAUCSL')
inflation = cpi.pct_change(periods = 12) * 100 
inflation.dropna(inplace = True)
inflation_df = pd.DataFrame(inflation, columns=['inflation'])
inflation_df = inflation_df.resample('ME').last()
inflation_df.index.name = 'date'
inflation_df = inflation_df[inflation_df.index >= '1986-02-28 00:00:00']
inflation_df.head(10)                                                                                                                                                                                                                                                   

In [ ]:
oil = fred.get_series('DCOILWTICO') 
unemployment = fred.get_series('UNRATE')    
m2 = fred.get_series('M2SL')

exo_df = pd.DataFrame({'oil': oil, 'unemployment': unemployment, 'm2': m2})
exo_df = exo_df.resample('ME').last()  
exo_df.index.name = 'date'

inflation_exo = pd.concat([inflation_df, exo_df], axis=1)
inflation_exo.ffill(inplace=True)
inflation_exo.dropna(inplace=True)

inflation_exo['oil_diff'] = inflation_exo['oil'].diff() 
inflation_exo['m2_diff'] = inflation_exo['m2'].diff()
inflation_exo.dropna(inplace=True) 

inflation_exo.head(10)

In [ ]:
train_size_no_exo = int(len(inflation_df) * 0.8)
train = inflation_df['inflation'][:train_size_no_exo]
test = inflation_df['inflation'][train_size_no_exo:] 

model_no_exo = SARIMAX(train, order = (1,1,1), seasonal_order = (2,1,1,12))
fit_no_exo = model_no_exo.fit(disp = False)


def forecast_no_exo(fit_no_exo, forecast_steps):
    forecast = fit_no_exo.forecast(steps=forecast_steps)   
    return forecast

forecast_steps = len(test)

forecast_no_exo = forecast_no_exo(fit_no_exo, forecast_steps)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(train.index, train, label = 'Training Data')
plt.plot(test.index, test, label = 'Testing Data')
plt.plot(forecast_no_exo.index, forecast_no_exo, label = 'Forecasted without Exogenous Variables')
plt.xlabel('Date')
plt.ylabel('Inflation Rate (%)')
plt.title('Inflation Rate Forecast without Exogenous Variables')
plt.legend()
plt.show()

In [ ]:
train_size_exo = int(len(inflation_exo)*0.8)
train_exo = inflation_exo.iloc[:train_size_exo]
test_exo = inflation_exo.iloc[train_size_exo:]

model_exo = SARIMAX(train_exo['inflation'], exog = train_exo[['oil_diff', 'unemployment', 'm2_diff']], order=(1,1,1), seasonal_order=(2,1,1,12), )
fit_exo = model_exo.fit(disp=False)

exog=test_exo[['oil_diff', 'unemployment', 'm2_diff']].iloc[:forecast_steps]

forecast_steps_exo = len(test_exo)

def forecast_exo(fit_exo, forecast_steps_exo, exog):
    forecast = fit_exo.forecast(steps=forecast_steps_exo, exog=exog)   
    return forecast

forecast_exo = forecast_exo(fit_exo, forecast_steps_exo, exog)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(train_exo.index, train_exo['inflation'], label = 'Training Data')
plt.plot(test_exo.index, test_exo['inflation'], label = 'Testing Data')
plt.plot(forecast_exo.index, forecast_exo, label = 'Forecast with Exogenous Variables')
plt.xlabel('Date')
plt.ylabel('Inflation Rate (%)')
plt.title('Inflation Rate Forecast with Exogenous Variables')
plt.legend()
plt.show()

In [ ]:
rmse_no_exo = np.sqrt(mean_squared_error(test.iloc[:forecast_steps], forecast_no_exo))
mae_no_exo = mean_absolute_error(test.iloc[:forecast_steps], forecast_no_exo)
print('Root Mean Squared Error: ', round(rmse_no_exo, 4))
print('Mean Absolute Error: ', round(mae_no_exo, 4))

In [ ]:
rmse_exo = np.sqrt(mean_squared_error(test_exo['inflation'].iloc[:forecast_steps_exo], forecast_no_exo))
mae_exo = mean_absolute_error(test_exo['inflation'].iloc[:forecast_steps_exo], forecast_no_exo)
print('Root Mean Squared Error: ', round(rmse_exo, 4))
print('Mean Absolute Error: ', round(mae_exo, 4))

In [ ]:
rmse_pct_change = ((rmse_no_exo - rmse_exo) / rmse_no_exo) * 100
mae_pct_change = ((mae_no_exo - mae_exo) / mae_no_exo) * 100

print(f'RMSE Percentage Change:  {round(rmse_pct_change, 2)}%')
print(f'MAE Percentage Change:  {round(mae_pct_change, 2)}%')